In [ ]:
using Distributed

In [ ]:
addprocs(2)

In [ ]:
@everywhere using DistributedArrays, DistributedJets, Jets, WaveFD, JetPackWaveFD, DistributedOperations, Schedulers

In [ ]:
v = read!("../02_marmousi_model_setup/marmousi_vp_20m_176x851.bin", Array{Float32}(undef, 176,851));
dz,dx = 20.0,20.0
nz,nx = size(v)
@show dz,dx
@show nz,nx;

In [ ]:
sx = range(0,length=4,stop=(851-1)*20)
nshots = length(sx)
@show nshots;

## Broadcast the velocity (v) to workers

In [ ]:
_v = bcast(v)

## Build a list of keyword arguments to be passed to the modeling operator

In [ ]:
kwargs = (reportinterval = 0, freqQ=5, srcfieldfile="", nsponge = 200)

## Note on scratch space for temporary files
When dealing with multiple shots as in this example, we need to specify the location where scratch files will be written.

You will need to change this to point to a temporary directory available on your system.

In [2]:
@everywhere scratch = "/mnt/scratch"
@assert isdir(scratch)

## Define function to be run by the Scheduler

In [ ]:
@everywhere function modelshot(isrc,sx,_v; kwargs...)
    nz,nx,dz,dx = 176,851,20.0,20.0   
    @info "modeling shot $(isrc) on $(gethostname()) with id $(myid())..."
    F = JopNlProp2DAcoIsoDenQ_DEO2_FDTD(;
        b = ones(Float32,nz,nx),
        nthreads = div(Sys.CPU_THREADS,2),
        isinterior = true,
        ntrec = 1101,
        dtrec = 0.002,
        dtmod = 0.001,
        dz = dz,
        dx = dx,
        wavelet = WaveletCausalRicker(f=5.0),
        sx = sx[isrc],
        sz = dz,
        rx = dx*[0:1:nx-1;],
        rz = 2*dz*ones(length(0:1:nx-1)),
        srcfieldfile = joinpath(scratch, "field-$i.$(randstring()).bin"),
        reportinterval=1000,
        kwargs...)
    
    d = F*_v
    @info "...done modeling shot $(isrc) on $(gethostname()) with id $(myid())"
    @info "extrema of shot $(isrc) is $(extrema(d))"

    # typically write to cloud storage here
    nothing
end

## Use cvxpmap to schedule tasks to workers (dynamic schedule)

In [ ]:
epmap(i->modelshot(i,sx,localpart(_v);kwargs...), 1:nshots)

## Remove workers

In [ ]:
rmprocs(workers())